In [1]:
import numpy as np
import os
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import glob
import shutil
from PIL import Image


# import the data

In [2]:
df=pd.read_excel('data.xlsx', index_col = 0)

In [6]:
df2=pd.read_csv('retinopathy_train.csv', index_col = 0)

In [7]:
#df = pd.read_csv('full_df.csv', index_col=0)
df.columns = df.columns.str.replace(' ','_').str.lower()
df.shape

(3500, 14)

In [8]:
df2.head()

,diagnosis
id_code,
000c1434d8d7,2
001639a390f0,4
0024cdab0c1e,1
002c21358ce6,0
005b95c28852,0


In [11]:
#explore data set, 0 is no diabetic retinopathy, 1-4 is severity
eda=df2.groupby('diagnosis').count()
eda

""
diagnosis
0
1
2
3
4


In [65]:
df.head()

,patient_age,patient_sex,left-fundus,right-fundus,left-diagnostic_keywords,right-diagnostic_keywords,n,d,g,c,a,h,m,o
ID,,,,,,,,,,,,,,
0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0
1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1
3,66,Male,3_left.jpg,3_right.jpg,normal fundus,branch retinal artery occlusion,0,0,0,0,0,0,0,1
4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1


In [68]:
df['left-fundus'].value_counts(ascending=False)

0_left.jpg       1
2960_left.jpg    1
2949_left.jpg    1
2950_left.jpg    1
2951_left.jpg    1
                ..
1243_left.jpg    1
1245_left.jpg    1
1246_left.jpg    1
1247_left.jpg    1
4784_left.jpg    1
Name: left-fundus, Length: 3500, dtype: int64

# Normal Eye Sample

In [69]:
normal_full=df[df['n'] == 1]

## Left Eye

In [70]:
#pull out only normal eyes
normal_left=normal_full[normal_full['left-diagnostic_keywords'].str.contains('normal fundus')]

In [71]:
normal_left.head()

,patient_age,patient_sex,left-fundus,right-fundus,left-diagnostic_keywords,right-diagnostic_keywords,n,d,g,c,a,h,m,o
ID,,,,,,,,,,,,,,
1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
8,59,Male,8_left.jpg,8_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
84,51,Female,84_left.jpg,84_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
191,51,Female,191_left.jpg,191_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
394,63,Male,394_left.jpg,394_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0


In [72]:
normal_left.shape

(1136, 14)

In [73]:
normal_left['left-fundus'].nunique()

1136

In [74]:
#checked to see how many unique key diagnostic labels
nl=normal_left.groupby(['left-diagnostic_keywords']).sum()
nl

,patient_age,n,d,g,c,a,h,m,o
left-diagnostic_keywords,,,,,,,,,
lens dust，normal fundus,4790,86,0,0,0,0,0,0,0
normal fundus,58045,1023,0,0,0,0,0,0,0
normal fundus，lens dust,1407,25,0,0,0,0,0,0,0
normal fundus，normal fundus,101,2,0,0,0,0,0,0,0


In [75]:
#to remove any other conditions, pure sample
normal_left2=normal_left[~normal_left['left-diagnostic_keywords'].str.contains('dust')]


In [76]:
#check to ensure only patients with normal eyes were included in list
al_check=normal_left2.groupby(['left-diagnostic_keywords']).sum()
al_check

,patient_age,n,d,g,c,a,h,m,o
left-diagnostic_keywords,,,,,,,,,
normal fundus,58045,1023,0,0,0,0,0,0,0
normal fundus，normal fundus,101,2,0,0,0,0,0,0,0


## Right Eye

In [77]:
#pull out only normal eyes
normal_right=normal_full[normal_full['right-diagnostic_keywords'].str.contains('normal fundus')]

In [78]:
#checked to see how many unique key diagnostic labels
nr=normal_right.groupby(['right-diagnostic_keywords']).sum()
nr

,patient_age,n,d,g,c,a,h,m,o
right-diagnostic_keywords,,,,,,,,,
lens dust，normal fundus,4835,88,0,0,0,0,0,0,0
normal fundus,58579,1033,0,0,0,0,0,0,0
normal fundus，lens dust,1080,18,0,0,0,0,0,0,0


In [79]:
#to remove any other conditions, pure sample
normal_right2=normal_right[~normal_right['right-diagnostic_keywords'].str.contains('dust')]

In [80]:
#check to ensure only patients with normal eyes were included in list
ar_check=normal_right2.groupby(['right-diagnostic_keywords']).sum()
ar_check

,patient_age,n,d,g,c,a,h,m,o
right-diagnostic_keywords,,,,,,,,,
normal fundus,58579,1033,0,0,0,0,0,0,0


In [81]:
#make a list of images for normal sample
normal_list_images=list(normal_left2['left-fundus'])+list(normal_right2['right-fundus'])
len(normal_list_images)

2058

# Diabetic Eyes

In [82]:
diabetic_full=df[df['d'] == 1]
diabetic_full.shape

(1128, 14)

## Left Eye

In [83]:
#pull out only diabetic eyes for left side
diabetic_left=diabetic_full[diabetic_full['left-diagnostic_keywords'].str.contains('diabetic retinopathy')]
diabetic_left.shape

(49, 14)

In [84]:
#checked to see how many unique key diagnostic labels
dl=diabetic_left.groupby(['left-diagnostic_keywords']).sum()
dl

,patient_age,n,d,g,c,a,h,m,o
left-diagnostic_keywords,,,,,,,,,
diabetic retinopathy,602,0,10,2,0,4,0,0,2
diabetic retinopathy，dry age-related macular degeneration,52,0,1,0,0,1,0,0,0
diabetic retinopathy，post laser photocoagulation,125,0,2,0,0,0,0,0,2
diabetic retinopathy，wet age-related macular degeneration,59,0,1,0,0,1,0,0,0
diabetic retinopathy，wet age-related macular degeneration，macular epiretinal membrane,42,0,1,0,0,1,0,0,1
dry age-related macular degeneration，diabetic retinopathy,185,0,3,0,0,3,0,0,0
epiretinal membrane，laser spot，severe proliferative diabetic retinopathy,38,0,1,0,0,0,0,0,1
glaucoma，diabetic retinopathy,359,0,6,6,0,0,0,0,0
"hypertensive retinopathy,diabetic retinopathy",55,0,1,0,0,0,1,0,1


In [85]:
#to remove any other conditions, pure sample
diabetic_left2=diabetic_left[~diabetic_left['left-diagnostic_keywords'].str.contains('atrophy|glaucoma|laser|dust|hypertensive|age-related')]


In [86]:
diabetic_other=diabetic_left2
diabetic_left2.shape

(21, 14)

In [87]:
#check to ensure only patients with diabetic eyes were included in list
dl_check=diabetic_left2.groupby(['left-diagnostic_keywords']).sum()
dl_check

,patient_age,n,d,g,c,a,h,m,o
left-diagnostic_keywords,,,,,,,,,
diabetic retinopathy,602,0,10,2,0,4,0,0,2
proliferative diabetic retinopathy,291,0,5,0,0,0,0,0,0
severe proliferative diabetic retinopathy,342,0,6,0,2,0,0,0,2


## Right Eye

In [88]:
#pull out only diabetic eyes (right side)
diabetic_right=diabetic_full[diabetic_full['right-diagnostic_keywords'].str.contains('diabetic retinopathy')]

In [89]:
diabetic_right['right-fundus'].nunique()

46

In [90]:
#checked to see how many unique key diagnostic labels
dr=diabetic_right.groupby(['right-diagnostic_keywords']).sum()
dr

,patient_age,n,d,g,c,a,h,m,o
right-diagnostic_keywords,,,,,,,,,
diabetic retinopathy,293,0,5,0,0,2,0,0,1
diabetic retinopathy，dry age-related macular degeneration,200,0,3,0,0,3,0,1,0
diabetic retinopathy，macular epiretinal membrane,42,0,1,0,0,1,0,0,1
diabetic retinopathy，post laser photocoagulation,63,0,1,0,0,0,0,0,1
dry age-related macular degeneration，diabetic retinopathy,209,0,3,0,0,3,0,0,0
glaucoma，diabetic retinopathy,337,0,6,6,0,0,0,0,0
hypertensive retinopathy，diabetic retinopathy,56,0,1,0,0,0,1,0,0
hypertensive retinopathy，suspected diabetic retinopathy,64,0,1,0,0,0,1,0,0
macular epiretinal membrane，diabetic retinopathy,80,0,1,0,0,0,0,0,1


In [91]:
#to remove any other conditions, pure sample
diabetic_right2=diabetic_right[~diabetic_right['right-diagnostic_keywords'].str.contains('atrophy|glaucoma|laser|dust|hypertensive|age-related|membrane|optic|white')]

In [92]:
diabetic_right2

,patient_age,patient_sex,left-fundus,right-fundus,left-diagnostic_keywords,right-diagnostic_keywords,n,d,g,c,a,h,m,o
ID,,,,,,,,,,,,,,
67,56,Male,67_left.jpg,67_right.jpg,diabetic retinopathy,diabetic retinopathy,0,1,0,0,0,0,0,0
234,54,Male,234_left.jpg,234_right.jpg,diabetic retinopathy,diabetic retinopathy,0,1,0,0,0,0,0,0
249,64,Male,249_left.jpg,249_right.jpg,moderate non proliferative retinopathy,proliferative diabetic retinopathy,0,1,0,0,0,0,0,0
412,60,Female,412_left.jpg,412_right.jpg,severe nonproliferative retinopathy,proliferative diabetic retinopathy,0,1,0,0,0,0,0,0
712,62,Male,712_left.jpg,712_right.jpg,wet age-related macular degeneration,diabetic retinopathy,0,1,0,0,1,0,0,0
978,59,Male,978_left.jpg,978_right.jpg,diabetic retinopathy，wet age-related macular d...,diabetic retinopathy,0,1,0,0,1,0,0,0
1838,62,Male,1838_left.jpg,1838_right.jpg,diabetic retinopathy，post laser photocoagulation,diabetic retinopathy,0,1,0,0,0,0,0,1
3994,70,Female,3994_left.jpg,3994_right.jpg,moderate non proliferative retinopathy,severe proliferative diabetic retinopathy,0,1,0,0,0,0,0,0
4002,67,Male,4002_left.jpg,4002_right.jpg,severe nonproliferative retinopathy,proliferative diabetic retinopathy,0,1,0,0,0,0,0,0


In [93]:
#check to ensure only patients with diabetic eyes were included in list
dr_check=diabetic_right2.groupby(['right-diagnostic_keywords']).sum()
dr_check

,patient_age,n,d,g,c,a,h,m,o
right-diagnostic_keywords,,,,,,,,,
diabetic retinopathy,293,0,5,0,0,2,0,0,1
proliferative diabetic retinopathy,515,0,9,0,0,0,0,0,0
severe proliferative diabetic retinopathy,70,0,1,0,0,0,0,0,0


In [95]:
dlist_left=list(diabetic_left2['left-fundus'])
#dlist_left

In [96]:
#make a list of images for diabetic sample
diabetic_list_images=list(diabetic_left2['left-fundus'])+list(diabetic_right2['right-fundus'])
len(diabetic_list_images)

36

In [98]:
#diabetic_list_images

# EDA

Normal (N),\
Diabetes (D),\
Glaucoma (G),\
Cataract (C),\
Age related Macular Degeneration (A),\
Hypertension (H),\
Pathological Myopia (M),\
Other diseases/abnormalities (O)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isna().count()

**Structure of numerical features**\
\
This process will give us some insights about the number of binary (2 unique values), \
ordinal (3 to ~10 unique values) and continuous (more than 10 unique values) features in the dataset.

In [ ]:
# For each numerical feature compute number of unique entries
unique_values = df.select_dtypes(include="number").nunique().sort_values()

# Plot information with y-axis in log-scale
unique_values.plot.bar(logy=True, figsize=(15, 4), title="Unique values per feature");

In [ ]:
df.describe()

## Heatmap-to see if any missing values

In [ ]:
plt.figure(figsize=(10, 8))
plt.imshow(df.isna(), aspect="auto", interpolation="nearest", cmap="gray")
plt.xlabel("Column Number")
plt.ylabel("Sample Number");

In [ ]:
# Plots the histogram for each numerical feature in a separate subplot
df.hist(bins=20, figsize=(15, 25), layout=(-1, 5), edgecolor="black")
plt.tight_layout();

In [ ]:
df.columns

In [ ]:
df.groupby(df['labels'])['patient_age'].count().plot.bar()


In [ ]:
df.groupby(df['labels'])['patient_sex'].count().plot.bar()

In [ ]:
sns.pairplot(
    df, plot_kws={"s": 50, "alpha": .4},hue = "patient_sex", palette="Set2",
    x_vars=["patient_age"],
    y_vars="labels",
    height = 7, aspect =1,
    );


In [ ]:
# Computes feature correlation
df_corr = df.corr(method="pearson")

In [ ]:
# Create labels for the correlation matrix
labels = np.where(np.abs(df_corr)>0.75, "S",
                  np.where(np.abs(df_corr)>0.5, "M",
                           np.where(np.abs(df_corr)>0.25, "W", "")))

# Plot correlation matrix
plt.figure(figsize=(15, 15))
sns.heatmap(df_corr, mask=np.eye(len(df_corr)), square=True,
            center=0, annot=labels, fmt='', linewidths=.5,
            cmap="vlag", cbar_kws={"shrink": 0.8});

## Pull diabetic info

I am interested in diabetic retinopathy therefore I will pull the data for normal eye and those diagnosed with diabetes

# Images

## Pull normal and diabetic images from pool of images

In [113]:
#move the normal images to normal folder and diabetic images to diabetic folder

src_dir = "/Users/karenbolon/Documents/spiced_projects/sandbox_dir/Project/images/"
dst_dir = "/Users/karenbolon/Documents/spiced_projects/sandbox_dir/Project/data/normal/"
dst_dir2 = "/Users/karenbolon/Documents/spiced_projects/sandbox_dir/Project/data/diabetic/"


In [114]:
for jpgfile in os.listdir(src_dir):
        if jpgfile in normal_list_images:
            shutil.copy(os.path.join(src_dir, jpgfile), dst_dir)

In [115]:
for jpgfile in os.listdir(src_dir):
        if jpgfile in diabetic_list_images:
            shutil.copy(os.path.join(src_dir, jpgfile), dst_dir2)

In [116]:
src_dir1 = "/Users/karenbolon/Documents/spiced_projects/sandbox_dir/Project/data/normal/"
src_dir2 = "/Users/karenbolon/Documents/spiced_projects/sandbox_dir/Project/data/diabetic/"
dst_dir3 = "/Users/karenbolon/Documents/spiced_projects/sandbox_dir/Project/test/normal/"
dst_dir4 = "/Users/karenbolon/Documents/spiced_projects/sandbox_dir/Project/test/diabetic/"


In [117]:
#to copy files over for test
normal_list = 1

for jpgfile in os.listdir(src_dir1):
    if normal_list <36 :
        shutil.copy(os.path.join(src_dir1, jpgfile), dst_dir3)
        normal_list += 1



In [118]:
#to copy files over for test
diabetic_list=1

for jpgfile in os.listdir(src_dir2):
    if diabetic_list <=36:
        shutil.copy(os.path.join(src_dir2, jpgfile), dst_dir4)        
        diabetic_list +=1
